### Replace uploaded files
Use this notebook when you want to replace a (set of) file(s) that had not yet been released.
Replacement consists in setting the correct file status, deleting some properties on the File item that refer to the previously uploaded file,
and deleting wfr items and related quality metrics that the previously uploaded file was input for.

The deletion of old file(s) from s3 is OPTIONAL and needs to be handled separately. See NOTE 2 below.

The upload of new file(s) is REQUIRED and needs to be handled separately, e.g. via Submit4DN.

The notebook checks the status of items before patching.

**NOTE 1:** only use this when the Files to replace are not yet released.

**NOTE 2:** you need to delete files from S3 if you don't immediately proceed with uploading a new file,
since the hourly md5 check reverts the effect of running this notebook.

In [ ]:
from dcicutils import ff_utils
from functions.notebook_functions import *

my_auth = get_key('andrea_data')

# indicate files to reset
files_acc = '''
4DNFIXXXXXX1
4DNFIXXXXXX2
'''

files_acc = [f for f in files_acc.split('\n') if f]
for file_accession in files_acc:
    assert file_accession.startswith('4DNFI'), f"{file_accession} is not a file accession"
print(f"{len(files_acc)} files")

In [ ]:
# load functions
def find_attached_items(file):
    """Given a File, list @id and status of all wfr inputs and linked quality metrics"""
    def _get_id_status(item):
        return item['@id'], item['status']

    if file['@type'][0] in ['FileFastq', 'FileProcessed']:
        wfr_inputs = file.get('workflow_run_inputs', [])
    else:
        # exception for File types that do not have workflow_run_inputs
        wfr_inputs = search_wfrs_with_input_file(file)

    item_ids = []
    for wfr_input in wfr_inputs:
        # append wfr that has the file as input
        item_ids.append(_get_id_status(wfr_input))
        wfr_input_object = ff_utils.get_metadata(wfr_input['@id'], key=my_auth)
        # append any qc of the wfr
        if wfr_input_object.get('quality_metric'):
            item_ids.append(_get_id_status(wfr_input_object['quality_metric']))

    return item_ids


def search_wfrs_with_input_file(file):
    """Returns wfr items that have a given file as input.
    This is helpful when handling FileReference or others that do not have workflow_run_inputs"""
    query = '/search/?type=WorkflowRunAwsem&input_files.value.accession=' + file['accession']
    wfr_items = ff_utils.search_metadata(query, key=my_auth)
    return wfr_items


def delete_item(item_id, status):
    """Delete item if status allows"""
    assert status in ['in review by lab', 'pre-release'], "Item status is incompatible with this change"
    if ACTION:
        res = ff_utils.patch_metadata({"status": "deleted"}, item_id, key=my_auth)
        if res.get('status') == 'success':
            print(f"Deleted {item_id}")
    else:
        print(f"{item_id} will be deleted")
    return


def delete_file_fields(file, delete_extrafiles=False):
    """Reset file to uploading if status allows"""
    assert file['status'] in ['uploaded', 'pre-release'], "File status is incompatible with this change"
    fields_to_be_removed = ['filename', 'md5sum', 'content_md5sum', 'file_size']
    if file.get('extra_files') and delete_extrafiles is True:
        assert len(file['extra_files']) == 1, "There is > 1 extra file, handle manually"
        fields_to_be_removed.append('extra_files')
    del_add_on = 'delete_fields=' + ','.join(fields_to_be_removed)

    if ACTION:
        res = ff_utils.patch_metadata({"status": "uploading"}, file['@id'], key=my_auth, add_on=del_add_on)
        if res.get('status') == 'success':
            print(f"Reset {file['@id']} to uploading")
        else:
            print(res)
    else:
        print(f"{file['@id']} will be reset to uploading")
    return

In [ ]:
# main

# set action to True to patch items, otherwise a dry run will be executed
ACTION = False

# this erases extra_files completely, if there is just one extra file
# use with caution
DELETE_EXTRAFILES = True

files_query = '/search/?type=File' + ''.join(['&accession=' + f for f in files_acc])
files_result = ff_utils.search_metadata(files_query, key=my_auth)
for file in files_result:
    # Find and delete items linked to the file
    additional_items = find_attached_items(file)
    for (item_id, status) in additional_items:
        delete_item(item_id, status)
    # Reset file
    delete_file_fields(file, DELETE_EXTRAFILES)